# はじめに
Deep Learning系の技術が流行っていますが、画像認識などの技術に比べて、機械翻訳や文書分類などの自然言語処理系の技術はとっつきにくいと考えられているようです。その原因の大部分を前処理のめんどくささが占めています。どのDeep LearningフレームワークにもLSTMなどのtextを扱うモデルのチュートリアルがあるのですが、現状

+ 簡単すぎるtoyモデルを題材としている
+ build in の前処理済みベンチマーク用データセットを題材としている
+ neural networkのコーディングに主眼を置おり、データ整備に関する記述は少ない
+ ほとんどが英語のデータを扱っており、日本語データを扱うチュートリアルは少ない

という問題点があり、公式のチュートリアルだけでは**「自分で用意した日本語データを使用して、Deep Learningモデルをパッと試してみる」**ことができるようになるまでギャップがある気がします。

そこで、この記事ではTorchtextというpythonライブラリを中心に、次のような内容を目指します。

+ さまざまな自然言語処理系のDeep Learningモデル作成に共通して必要なデータ処理フローを整理する。
+ torchtextを利用してそれらのデータ処理が少ないコーディング量で可能になることを説明する。
+ 英語だけではなく日本語のtextデータも扱える方法を解説する。
+ 読者が**自分で用意したtextデータでDLモデルをさくっと試す**ことが出来るようになる。

# Torchtext とは
torchtext とは自然言語処理関連の前処理を簡単にやってくれる非常に優秀なライブラリです。自分も業務で自然言語処理がからむDeep Learningモデルを構築するときなど大変お世話になっています。torchとついていますが、Pytorchからだけではなく、Tensorflowなどの他のライブラリと一緒に使うこともできます。

torchtextは強力なライブラリなのですが、英語も含めてdocumentは充実していません。既存の記事だと

+ [A Tutorial on Torchtext](http://anie.me/On-Torchtext/)
+ [TorchTextTutorial](https://github.com/mjc92/TorchTextTutorial/blob/master/01.%20Getting%20started.ipynb)
+ [torchtextのソースコードを読んでみた](http://hacks.deeplearning.jp/torchtext/)

が参考になります。

# textデータをDeep Learningモデルに入力する際の典型的なデータフロー
自然言語処理のタスクは、文書分類や機械翻訳などたくさんあります。Deep Learningのフレームワークもpytorchやtensorflowなどたくさんあります。しかし、どのタスクをどのフレームワークであつかう場合もtextデータをDeep Neural Networkに入力する際は、たいてい
1. センテンスを単語ごとに区切り、
2. 単語に番号を振って、
3. センテンスを表す番号の列をミニバッチごとにまとめた行列の形
にして渡す必要があります。

![flow.png](https://qiita-image-store.s3.amazonaws.com/0/183955/ff027e0f-7cd1-6a76-0eaa-05b8ae61074e.png)


この図の0:テキストデータ読み込み～4:単語埋め込みまでのデータフローの管理をしてくれるのがtorchtextです。
単語埋め込みの部分はDeep learningフレームワークの方で扱う場合が多いですが、学習済み埋め込みベクトルの管理はtorchtextで行えます。

つまり、テキストデータを読み込んで、ミニバッチごとに上図の4の形式の行列を出力するようなiteratorを作成するのがtorchtextの目標です。

以降の章で、上の図の各ステップをtorchtextでどのように記述するのか、説明していきます。

In [1]:
# coding:utf-8
import janome
from janome.tokenizer import Tokenizer

import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import FastText

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch.autograd import Variable

torch.manual_seed(0)
torch.cuda.manual_seed(0)

# 使用データ

train_ja.tsv
```
あなたを愛しています。	1
私はマイクが嫌いです。	0
私はマキがすきです。	1
私はあなたが好きではありません。	0
```
1カラム目がテキスト、2カラム目がラベル(0スタート)のtsvファイルを用意します。

# 手順0, 1:データの読み込みと分かち書き
まず初めにデータを読み込んで、単語に分割(tokenize)する処理に関して説明します。データのtokenizationにかかわってくるのがFieldクラスです。

## Fieldクラス
Fieldクラスは読み込んだデータに施す前処理とその結果を管理するクラスです。
torchtextでは、データを取り込む際、

1. 各データソースの各カラムに対して前処理を管理するFieldクラスを指定
2. 各カラムごとに、指定されたFieldクラスが管理する前処理が実行される

という流れになります。Fieldクラスが管理する前処理にはテキストのtokenizationも含まれます。

文書分類モデルだと、次のようにtext用のTEXTフィールドとラベル用のLABELフィールドを用意します。

In [2]:
j_t = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_t.tokenize(text, wakati=True)]

TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)

data.Fieldの各引数は次のような意味を持ちます。

+ sequential: 対応するデータがtextのように可変長のデータかどうか
+ lower: 文字をすべて小文字に変換するかどうか
+ tokenize: tokenizeや前処理に使用する関数
+ include_length: テキストの長さデータを保持するか
+ データ数方向の次元を1番目にもってくるか

tokenizeに使用する関数は文字列を受け取り、tokenize結果の配列を返す関数です。  
ここに記述した処理が、データを読み込んだ時に各カラムの各行に対して適用されます。  
次のよな挙動を想定します。

In [3]:
print(tokenizer(u'あなたを愛しています。'))

['あなた', 'を', '愛し', 'て', 'い', 'ます', '。']


### textデータのクリーニングに関して
textデータのクリーニングに関しては次の記事が参考になります。
[自然言語処理における前処理の種類とその威力](https://qiita.com/Hironsan/items/2466fe0f344115aff177)
この記事で記述されているクリーニング系の処理のうち、

+ 英語の大文字小文字変換
+ 出現頻度の低い単語の除去

はtorchtextの機能として実装されています。英語の大文字小文字変換は上のようにFieldのlower引数によって制御します。出現頻度の低い単語の除去は後の章で説明します。
その他のクリーニング処理はあらかじめ入力tsvファイル作成時に行ってもよいですが、htmlタグの除去などは次のようにFieldオブジェクトのtokenizer関数内に記述し、データの読み込みと同時に行うこともできます。

In [4]:
from bs4 import BeautifulSoup

def clean_tokenizer(text):
    soup = BeautifulSoup(text,"lxml")
    clean_text = soup.get_text()
    return [tok for tok in j_t.tokenize(clean_text, wakati=True)]

print(clean_tokenizer(u'<p>あなたを愛しています。</p>'))

['あなた', 'を', '愛し', 'て', 'い', 'ます', '。']


## データの読み込み
データパスと各カラムに対応するFieldを指定してデータを読み込みます。

In [5]:
train, val, test = data.TabularDataset.splits(
        path='./', train='train_ja.tsv',
        validation='val_ja.tsv', test='test_ja.tsv', format='tsv',
        fields=[('Text', TEXT), ('Label', LABEL)])

data.TabularDataset.splitsはデータを読み込んで、datasetオブジェクトを返します。  
このような形でfield引数にFieldオブジェクトを渡すと、train_ja.tsvの1列目にTEXT Fieldの、2列目にLABEL Fieldのtokenizerに記述された前処理＋単語分割処理が適用されます。そして、それぞれ返り値のdatasetオブジェクトに'Text','Label'という名前で格納されます。
結果は次のように確認できます。

In [6]:
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

len(train) 4
vars(train[0]) {'Text': ['あなた', 'を', 'が', '好き', 'です', '。'], 'Label': '1'}


# 手順2:単語へ番号を振る
読み込んだデータに出現した単語のリストを作成し、単語に番号を振ります。Fieldクラスのbuild_vocabメソッドを使用します。

In [7]:
TEXT.build_vocab(train, min_freq=2)

上記のコードでは次のような処理が行われます。

1. データ内の各単語の数をカウントし、TEXT.vocab.freqsに格納
2. min_freq以上の回数出現した単語に番号を振り、番号から単語への辞書をTEXT.vocab.itosに、単語から番号への辞書をTEXT.vocab.stoiに格納

単語カウントや番号を振った結果は次のように確認できます。

単語カウント結果

In [8]:
TEXT.vocab.freqs

Counter({'。': 4,
         'あなた': 1,
         'が': 4,
         'です': 4,
         'は': 2,
         'を': 1,
         'ボブ': 1,
         'マイク': 1,
         'マキ': 1,
         '好き': 2,
         '嫌い': 2,
         '私': 2})

単語-番号辞書

In [9]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<pad>': 1,
             '<unk>': 0,
             '。': 2,
             'が': 3,
             'です': 4,
             'は': 5,
             '好き': 6,
             '嫌い': 7,
             '私': 8})

min_freqを2としたので、出現回数が2未満の単語は未知語unk扱いになります。
padはバッチ作成時に使用するダミートークンで、後の章で説明します。

番号-単語

In [10]:
TEXT.vocab.itos

['<unk>', '<pad>', '。', 'が', 'です', 'は', '好き', '嫌い', '私']

### 学習済み単語ベクトルについて
build_vocabメソッドのvectors引数を使用すると、単語へ番号を振るのと同時に、学習済みの単語ベクトルを指定し、読み込むことができます。
単語ベクトルは、その言語のすべての単語について保持しておく必要はなく、学習データにおける出現回数がmin_freq以上の単語に対応するベクトルのみメモリ上に保持します。そのため、torchtextでは上記のように単語の出現回数のカウントを行うbuild_vocabメソッドで学習済みの単語ベクトルを読み込んでいます。
独自のベクトル群を指定することもできれば、webで公開されている学習済みベクトルをダウンロードしてくることも可能です。公開されている単語ベクトルの内torchtextが対応しているものは[glove](https://nlp.stanford.edu/projects/glove/)と[fasttext](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md)と[charngram](http://www.logos.t.u-tokyo.ac.jp/~hassy/publications/arxiv2016jmt/)です。このうち、日本語にも対応しているのはfasttextです。fasttextのベクトルをダウンロードして来て、読み込むには

In [11]:
TEXT.build_vocab(train, vectors=FastText(language="ja"), min_freq=2)

とします。
このように指定すると、単語に番号を振る処理に加えて

1. .vector_cache/以下をチェック
2. ダウンロード済みでなければ.vector_cache/以下にfasttextで学習したベクトル群をダウンロード
3. trainにおける出現回数がmin_freq以上の単語に関して、学習済み単語ベクトルを読み込みTEXT.vocab.vectorsに格納

という処理が走ります。
TEXT.vocab.vectorsは

+ 行数:min_freq以上出現した単語数+特殊トークン数(\<unk\> , \<pad\>)
+ 列数:単語ベクトルの次元。デフォルトだとfasttextは300次元

という行列になります。

In [12]:
TEXT.vocab.vectors.size()

torch.Size([9, 300])

学習済みの単語ベクトルを使用しない場合はvectorsを指定しません。

# 手順3:バッチ化

data.Iterator.splits はdatasetオブジェクトから、各単語を番号に変換してミニバッチごとにまとめた行列を返すイテレータを作成できます。

In [13]:
train_iter, val_iter, test_iter = data.Iterator.splits(
        (train, val, test), batch_sizes=(2, 2, 1), device=-1, repeat=False,sort=False)

イテレータが返す結果は次のように確認できます。

In [14]:
batch = next(iter(train_iter))
print(batch.Text)
print(batch.Label)

(Variable containing:
    0     0     3     6     4     2     1
    8     5     0     3     7     4     2
[torch.LongTensor of size 2x7]
, 
 6
 7
[torch.LongTensor of size 2]
)
Variable containing:
 1
 0
[torch.LongTensor of size 2]



batch.Textは単語番号の行列と、テキストデータの長さを表す配列のタプルになっています。(data.Fieldの引数でinclude_length=Trueとしたためです)RNNなどの学習で[masked BPTT](https://en.wikipedia.org/wiki/Backpropagation_through_time)を使用する際は、テキストデータのミニバッチと同時に、このテキストの長さのデータを渡します。
学習ループでは、このbatch.Textを入力として、Deep Learningモデルによる予測を行い、予測とbatch.Labelを比べて誤差を計算して、学習アルゴリズムを回すことになります。

手順2で作成した番号-単語辞書でbatch.Textを変換すると  

+ 私 は あなた が unk unk は unk unk unk 。
+ 私 は unk が unk です 。 pad pad pad pad

となります。padトークンはこのようにバッチ化の際、短いtextに追加して、長さをそろえるために使用されます。デフォルトではミニバッチ内で最も長いtextの長さになるまでpaddingされます。

# 手順4:単語埋め込みベクトルの使用

手順2で読み込んだ単語埋め込みベクトルは、手順3で作成したイテレータが返す行列内の単語番号をベクトルに変換する際に使用されます。このように単語をベクトル化した結果がLSTMモジュールなどへの直接の入力となります。
単語番号をベクトルに変換するモジュールは各Deep Learningフレームワークに用意されているはずです。(pytorchならnn.Embedding、kerasならlayers.embeddings.Embeddingを使用します。)
例えばpytorchの場合はnn.Embeddingモジュールに次のように埋め込みベクトル(TEXT.vocab.vectors)を渡します。

In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, emb_dim, h_dim, v_size, gpu=True, batch_first=True):
        super(EncoderRNN, self).__init__()
        self.gpu = gpu
        self.h_dim = h_dim
        self.embed = nn.Embedding(v_size, emb_dim)
        self.embed.weight.data.copy_(TEXT.vocab.vectors)
        self.lstm = nn.LSTM(emb_dim, h_dim, batch_first=batch_first,
                            bidirectional=True)

# 手順5:attentionつき判別モデルの作成

データの準備が整ったので、文書分類モデルを構築していきます。attentionという機構を組み込んで、予測理由の可視化も行います。

# attentionの復習
attentionとは（正確な定義ではないですが）予測モデルに入力データのどの部分に注目するか知らせる機構のことです。
attention技術は機械翻訳への応用が特に有名です。
例えば、日英翻訳モデルを考えます。翻訳モデルは”これはペンです”という文字列を入力として"This is a pen"という英文を出力しますが、「pen」という文字を出力する際、モデルは入力文の「ペン」という文字に注目するはずです。このように入力データのある部分に「注目する=attention」という機構を予測モデルに組み込むことで、種々のタスクにおいいて精度が向上することが報告されてきました。
また、このattentionを可視化することで「入力データのどの部分に注目して予測を行ったか」という形で予測理由の提示を行うことができます。
attentionについての説明と実装は

+ [pytorch チュートリアル](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb)

がとても参考になります。

# self attention を利用した分類
今回は、attentionの技術を利用して、予測理由が可視化できる文書分類モデルを実装していきます。
[self-attentive sentence embedding](https://arxiv.org/pdf/1703.03130.pdf) という論文の手法を単純化したものになります。
この手法は次のような手順で予測を行います。

1. bidirectional LSTMで文書を変換
2. 各単語に対応する隠れ層(下図$h_i$)を入力とし、予測の際その単語に注目すべき確率（self attention 下図$A_i$）をNeural Networkで予測
3. self attention の重み付で各単語に対応する隠れ層を足し合わせたものを入力とし、Neural Networkで文書のラベルを予測

この$A_i$を可視化してやれば、モデルが予測の際どの単語に注目したかを知ることができます。
(オリジナル論文では複数個のself attentionを利用する方法が提案されているのですが、今回は簡易のためattentionは1種類としています。)

![image.png](https://qiita-image-store.s3.amazonaws.com/0/183955/8e7d33eb-6182-81c9-7eca-cc138e5f1e02.png)


bidirectional lstmの部分

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, emb_dim, h_dim, v_size, gpu=True, v_vec=None, batch_first=True):
        super(EncoderRNN, self).__init__()
        self.gpu = gpu
        self.h_dim = h_dim
        self.embed = nn.Embedding(v_size, emb_dim)
        if v_vec is not None:
            self.embed.weight.data.copy_(v_vec)
        self.lstm = nn.LSTM(emb_dim, h_dim, batch_first=batch_first,
                            bidirectional=True)

    def init_hidden(self, b_size):
        h0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        c0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        if self.gpu:
            h0 = h0.cuda()
            c0 = c0.cuda()
        return (h0, c0)

    def forward(self, sentence, lengths=None):
        self.hidden = self.init_hidden(sentence.size(0))
        emb = self.embed(sentence)
        packed_emb = emb

        if lengths is not None:
            lengths = lengths.view(-1).tolist()
            packed_emb = nn.utils.rnn.pack_padded_sequence(emb, lengths)

        out, hidden = self.lstm(packed_emb, self.hidden)

        if lengths is not None:
            out = nn.utils.rnn.pad_packed_sequence(output)[0]

        out = out[:, :, :self.h_dim] + out[:, :, self.h_dim:]

        return out

attentionクラスです。
LSTMの隠れ層を入力として、各単語へのattentionを出力します。

In [17]:
class Attn(nn.Module):
    def __init__(self, h_dim):
        super(Attn, self).__init__()
        self.h_dim = h_dim
        self.main = nn.Sequential(
            nn.Linear(h_dim, 24),
            nn.ReLU(True),
            nn.Linear(24,1)
        )

    def forward(self, encoder_outputs):
        b_size = encoder_outputs.size(0)
        attn_ene = self.main(encoder_outputs.view(-1, self.h_dim)) # (b, s, h) -> (b * s, 1)
        return F.softmax(attn_ene.view(b_size, -1), dim=1).unsqueeze(2) # (b*s, 1) -> (b, s, 1)

最後にattentionを利用して実際に文書分類を行う部分です。

In [18]:
class AttnClassifier(nn.Module):
    def __init__(self, h_dim, c_num):
        super(AttnClassifier, self).__init__()
        self.attn = Attn(h_dim)
        self.main = nn.Linear(h_dim, c_num)


    def forward(self, encoder_outputs):
        attns = self.attn(encoder_outputs) #(b, s, 1)
        feats = (encoder_outputs * attns).sum(dim=1) # (b, s, h) -> (b, h)
        return F.log_softmax(self.main(feats),dim=1), attns

学習・検証を行う関数

In [19]:
def train_model(epoch, train_iter, optimizer, log_interval=1, batch_size=2):
    encoder.train()
    classifier.train()
    correct = 0
    for idx, batch in enumerate(train_iter):
        (x, x_l), y = batch.Text, batch.Label
        optimizer.zero_grad()
        encoder_outputs = encoder(x)
        output, attn = classifier(encoder_outputs)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).cpu().sum()
        if idx % log_interval == 0:
            print('train epoch: {} [{}/{}], acc:{}, loss:{}'.format(
            epoch, (idx+1)*len(x), len(train_iter)*batch_size,
            correct/float(log_interval * len(x)),
            loss.data[0]))
            correct = 0

            
def test_model(epoch, test_iter):
    encoder.eval()
    classifier.eval()
    correct = 0
    for idx, batch in enumerate(test_iter):
        (x, x_l), y = batch.Text, batch.Label
        encoder_outputs = encoder(x)
        output, attn = classifier(encoder_outputs)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).cpu().sum()
    print('test epoch:{}, acc:{}'.format(epoch, correct/float(len(test))))

# 学習実行
ハイパーパラメータなどを指定して学習を実行します。

In [24]:
emb_dim = 300 #単語埋め込み次元
h_dim = 3 #lstmの隠れ層の次元
class_num = 2 #予測クラウ数
lr = 0.001 #学習係数
epochs = 100 #エポック数

 # make model
encoder = EncoderRNN(emb_dim, h_dim, len(TEXT.vocab),gpu=False, v_vec = TEXT.vocab.vectors)
classifier = AttnClassifier(h_dim, class_num)

# init model
def weights_init(m):
    classname = m.__class__.__name__
    if hasattr(m, 'weight') and (classname.find('Embedding') == -1):
        nn.init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))

for m in encoder.modules():
    print(m.__class__.__name__)
    weights_init(m)
    
for m in classifier.modules():
    print(m.__class__.__name__)
    weights_init(m)

# optim
from itertools import chain
optimizer = optim.Adam(chain(encoder.parameters(),classifier.parameters()), lr=lr)


EncoderRNN
Embedding
LSTM
AttnClassifier
Attn
Sequential
Linear
ReLU
Linear
Linear


In [25]:
# train model
for epoch in range(epochs):
    train_model(epoch + 1, train_iter, optimizer)
    test_model(epoch + 1, val_iter)

train epoch: 1 [2/4], acc:1.0, loss:0.560247540473938
train epoch: 1 [4/4], acc:0.5, loss:1.0654399394989014


/home/itok/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """
/home/itok/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


test epoch:1, acc:0.75
train epoch: 2 [2/4], acc:1.0, loss:0.5486541986465454
train epoch: 2 [4/4], acc:0.5, loss:0.8221700191497803
test epoch:2, acc:0.5
train epoch: 3 [2/4], acc:0.5, loss:0.5928575992584229
train epoch: 3 [4/4], acc:0.5, loss:0.6966612339019775
test epoch:3, acc:0.5
train epoch: 4 [2/4], acc:0.5, loss:0.6245954036712646
train epoch: 4 [4/4], acc:0.5, loss:0.7563215494155884
test epoch:4, acc:0.5
train epoch: 5 [2/4], acc:0.5, loss:0.5691448450088501
train epoch: 5 [4/4], acc:0.5, loss:0.730218768119812
test epoch:5, acc:0.5
train epoch: 6 [2/4], acc:0.5, loss:0.6600370407104492
train epoch: 6 [4/4], acc:0.5, loss:0.6107431650161743
test epoch:6, acc:0.5
train epoch: 7 [2/4], acc:1.0, loss:0.47765815258026123
train epoch: 7 [4/4], acc:0.0, loss:0.7952502965927124
test epoch:7, acc:0.5
train epoch: 8 [2/4], acc:0.5, loss:0.5760264992713928
train epoch: 8 [4/4], acc:1.0, loss:0.6201963424682617
test epoch:8, acc:0.75
train epoch: 9 [2/4], acc:1.0, loss:0.52674621343612

train epoch: 72 [2/4], acc:1.0, loss:0.1633222997188568
train epoch: 72 [4/4], acc:1.0, loss:0.2283763885498047
test epoch:72, acc:1.0
train epoch: 73 [2/4], acc:1.0, loss:0.12003561854362488
train epoch: 73 [4/4], acc:1.0, loss:0.2560301423072815
test epoch:73, acc:1.0
train epoch: 74 [2/4], acc:1.0, loss:0.15971876680850983
train epoch: 74 [4/4], acc:1.0, loss:0.21789105236530304
test epoch:74, acc:1.0
train epoch: 75 [2/4], acc:1.0, loss:0.19169685244560242
train epoch: 75 [4/4], acc:1.0, loss:0.17677609622478485
test epoch:75, acc:1.0
train epoch: 76 [2/4], acc:1.0, loss:0.2084178626537323
train epoch: 76 [4/4], acc:1.0, loss:0.1558011770248413
test epoch:76, acc:1.0
train epoch: 77 [2/4], acc:1.0, loss:0.24056687951087952
train epoch: 77 [4/4], acc:1.0, loss:0.10514883697032928
test epoch:77, acc:1.0
train epoch: 78 [2/4], acc:1.0, loss:0.19618894159793854
train epoch: 78 [4/4], acc:1.0, loss:0.15393829345703125
test epoch:78, acc:1.0
train epoch: 79 [2/4], acc:1.0, loss:0.2305940

# attentionの可視化

spanタグを利用して、attentionを可視化します。

In [26]:
from IPython.display import HTML

# 単語とattentionの強さを受け取ってspanタグをつける関数
def highlight(word, attn):
    html_color = u'#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
    return u'<span style="background-color: {}">{}</span>'.format(html_color, word)

# 単語番号と対応するattentionの配列を受け取って、spanタグで色付けされた文字列を返す関数
def mk_html(sentence, attns):
    def itos(word):
        word = TEXT.vocab.itos[word]
        return word.strip("<").strip(">")
    html = u""
    for word, attn in zip(sentence, attns):
        html += u' ' + highlight(
        itos(word),
        attn
        )
    return html + u"<br><br>"

In [27]:
res = ""
for batch in test_iter:
    x = batch.Text[0]
    y = batch.Label
    encoder_outputs = encoder(x)
    output, attn = classifier(encoder_outputs)
    pred = output.data.max(1, keepdim=True)[1]
    a = attn.data[0,:,0]
    res += u"正解{}:予測{}".format(str(y[0].data[0]), str(pred[0][0])) + mk_html(x.data[0], a)
HTML(res)

/home/itok/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  
